In [18]:
from sklearnex import patch_sklearn
patch_sklearn()

import pandas as pd
import seaborn as sns
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import make_scorer
import time
from sklearn.preprocessing import LabelEncoder

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [19]:
df = pd.read_csv('train_filtered2.csv')

In [20]:
df['date'] = pd.to_datetime(df['date'])

In [21]:
if 'cluster_id' in df.columns:
    le = LabelEncoder()
    df['cluster_id'] = le.fit_transform(df['cluster_id'])

In [22]:
df["month"] = df["date"].dt.month
df["dayofweek"] = df["date"].dt.dayofweek
df["quarter"] = df["date"].dt.quarter
df["day"] = df["date"].dt.day
df["weekofyear"] = df["date"].dt.isocalendar().week.astype(int)

In [23]:
df = df.drop(columns=["date"])

In [24]:
df.head()

,ID,cluster_id,electricity_consumption,temperature_2m_max,temperature_2m_min,sunshine_duration,daylight_duration,wind_speed_10m_max,wind_direction_10m_dominant,month,dayofweek,quarter,day,weekofyear
0,cluster_1_2014-01-01,0,358.032,10.8,4.2,53.003333,29787.533333,40.6,186.000000,1,2,1,1,1
1,cluster_2_2014-01-01,1,548.247,12.2,4.3,8195.656667,30650.350000,36.0,170.591118,1,2,1,1,1
2,cluster_3_2014-01-01,2,758.303,12.9,-0.8,16305.260000,31547.686667,20.9,159.467752,1,2,1,1,1
3,cluster_4_2014-01-01,3,1072.077,10.8,4.7,9224.803333,30769.220000,34.7,184.339753,1,2,1,1,1
4,cluster_1_2014-01-02,0,386.908,10.7,7.0,22372.000000,29850.226667,34.3,210.333465,1,3,1,2,1


In [25]:
target_col = "electricity_consumption"

In [26]:
df_with_cluster = df.copy()
df_without_cluster = df.drop(columns=["cluster_id"]) if "cluster_id" in df.columns else df.copy()

In [27]:
df_without_cluster.head()

,ID,electricity_consumption,temperature_2m_max,temperature_2m_min,sunshine_duration,daylight_duration,wind_speed_10m_max,wind_direction_10m_dominant,month,dayofweek,quarter,day,weekofyear
0,cluster_1_2014-01-01,358.032,10.8,4.2,53.003333,29787.533333,40.6,186.000000,1,2,1,1,1
1,cluster_2_2014-01-01,548.247,12.2,4.3,8195.656667,30650.350000,36.0,170.591118,1,2,1,1,1
2,cluster_3_2014-01-01,758.303,12.9,-0.8,16305.260000,31547.686667,20.9,159.467752,1,2,1,1,1
3,cluster_4_2014-01-01,1072.077,10.8,4.7,9224.803333,30769.220000,34.7,184.339753,1,2,1,1,1
4,cluster_1_2014-01-02,386.908,10.7,7.0,22372.000000,29850.226667,34.3,210.333465,1,3,1,2,1


In [28]:
def train_and_eval(df_input, label):
    X = df_input.drop(columns=[target_col])
    y = df_input[target_col]
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    model = XGBRegressor(objective="reg:squarederror", n_estimators=300, learning_rate=0.1, max_depth=5)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    print(f"✅ {label} RMSE: {rmse:.4f}")
    return rmse


In [29]:
df_without_cluster = df_without_cluster.drop(columns=["ID"])
print("Evaluating WITHOUT cluster_id...")
rmse_no_cluster = train_and_eval(df_without_cluster, "Without cluster_id")

Evaluating WITHOUT cluster_id...


✅ Without cluster_id RMSE: 220.7772


In [30]:
df_with_cluster = df_with_cluster.drop(columns=["ID"])
print("\nEvaluating WITH cluster_id...")
rmse_with_cluster = train_and_eval(df_with_cluster, "With cluster_id")


Evaluating WITH cluster_id...
✅ With cluster_id RMSE: 28.5746
